In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import joblib
import os

In [2]:
data = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/data"

### Check total expression file

In [3]:
expression_file = os.path.join(data, "raw_counts_gene_by_cell.tsv")
#read expression mtx
expression_mtx = pd.read_csv(expression_file, sep="\t")
#check the mtx
expression_mtx.head()
#check the shape of the mtx
expression_mtx.shape

(16368, 2089)

In [7]:
display(expression_mtx)

,Unnamed: 0,S4_AAACCCAAGCACGATG-1,S4_AAACCCAGTGAATAAC-1,S4_AAACCCAGTGGAGGTT-1,S4_AAACGAAGTGAGGAAA-1,S4_AAACGAAGTGTAACGG-1,S4_AAACGCTCATATAGCC-1,S4_AAACGCTCATCGATCA-1,S4_AAACGCTGTTAGAGAT-1,S4_AAAGAACAGTAAACTG-1,...,S4_TTTGATCAGTAATACG-1,S4_TTTGATCCATAGTCAC-1,S4_TTTGATCGTACTAACC-1,S4_TTTGGAGAGAGTCAAT-1,S4_TTTGGAGCAAGAGTTA-1,S4_TTTGGAGGTCTAACGT-1,S4_TTTGGAGTCTACTGCC-1,S4_TTTGGTTAGACCTTTG-1,S4_TTTGGTTAGCATCAAA-1,S4_TTTGTTGCAGTTGGTT-1
0,Mrpl15,3.0,11.0,2.0,17.0,2.0,3.0,8.0,0.0,9.0,...,4.0,2.0,5.0,5.0,2.0,15.0,4.0,2.0,1.0,2.0
1,Lypla1,0.0,1.0,0.0,4.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0
2,Gm37988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Tcea1,1.0,0.0,1.0,5.0,0.0,1.0,4.0,0.0,3.0,...,2.0,0.0,2.0,4.0,4.0,4.0,2.0,0.0,1.0,0.0
4,Atp6v1h,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16363,Fbxo15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16364,Neto1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16365,Naaladl1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16366,Gm26902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# save as .tsv.gz
expression_mtx.to_csv(os.path.join(data, "expression.tsv.gz"), sep="\t", compression="gzip")

### Prep base GRN

In [ ]:
binlinking_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added_v2/tmp_dynamic/Subset1/binlinking.tsv.gz"
#expression_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added_v2/tmp_dynamic/Subset1/expression.tsv.gz"
# read binlinking file
binlinking = pd.read_csv(binlinking_file, sep="\t")

In [5]:
display(binlinking)

,Unnamed: 0,A1BG,A1BG-AS1,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,...,ZSWIM8,ZSWIM9,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1
0,AHR,True,True,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False
1,ARID3A,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,ARID3B,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,ARID5A,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,ARID5B,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,ZNF860,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
478,ZNF891,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
479,ZNF90,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
480,ZNF98,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [6]:
base_grn_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/Data/grn_files/base_GRN.csv"
base_grn = pd.read_csv(base_grn_file)
display(base_grn)

,Unnamed: 0,peak_id,gene_short_name,9430076c15rik,Ac002126.6,Ac012531.1,Ac226150.2,Afp,Ahr,Ahrr,...,Znf784,Znf8,Znf816,Znf85,Zscan10,Zscan16,Zscan22,Zscan26,Zscan31,Zscan4
0,0,chr10_100050979_100052296,4930430F08Rik,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,chr10_101006922_101007748,SNORA17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,chr10_101144061_101145000,Mgat4c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,chr10_10148873_10149183,9130014G24Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,chr10_10149425_10149815,9130014G24Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91971,91971,chrX_99839037_99840900,Phka1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
91972,91972,chrY_345823_347369,Eif2s3y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91973,91973,chrY_581272_582790,Uty,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91974,91974,chrY_621772_623366,Ddx3y,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
collapsed_grn = base_grn.groupby('gene_short_name').max().reset_index()
#drop columns that are not needed
collapsed_grn = collapsed_grn.drop(columns=['Unnamed: 0', 'peak_id'])
# convert to boolean df (True/False) instead of 1/0
binlinking_mouse_base = (collapsed_grn.set_index('gene_short_name') > 0).reset_index()
#transpose the df
binlinking_mouse_base = binlinking_mouse_base.T
# Reset index to convert it to a column
binlinking_mouse_base = binlinking_mouse_base.reset_index()
# Set the first row as column headers
binlinking_mouse_base.columns = binlinking_mouse_base.iloc[0]
# Remove the first row
binlinking_mouse_base = binlinking_mouse_base[1:].reset_index(drop=True)
# Rename just the first column to empty string, keep others as they are
binlinking_mouse_base.columns = [''] + list(binlinking_mouse_base.columns[1:])
display(binlinking_mouse_base)


,,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007P08Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,0610009B22Rik,0610009D07Rik,...,snoR38,snoU105B,snoU109,snoU13,snoU54,snoU6-77,snoU85,snoU89,snoZ39,snosnR60_Z15
0,9430076c15rik,True,False,False,False,False,False,False,True,True,...,False,True,True,True,False,True,False,True,False,False
1,Ac002126.6,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
2,Ac012531.1,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,False
3,Ac226150.2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Afp,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,Zscan16,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1089,Zscan22,True,True,False,False,False,False,False,True,False,...,False,True,False,False,True,True,False,False,False,False
1090,Zscan26,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1091,Zscan31,False,False,False,False,True,False,False,True,True,...,True,False,False,True,False,True,False,False,False,False


In [19]:
#save as .csv
binlinking_mouse_base.to_csv(os.path.join(data, "mouse_base_grn.tsv.gz"), sep="\t", index=False)